In [2]:
from fastai.vision.all import *

In [3]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('../data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

# Foundations version
## Basic architecture

In [4]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [5]:
# num hidden
nh = 50

In [6]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.zeros(1)

In [7]:
def lin(x, w, b): return x@w+b

In [8]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [9]:
def relu(x): return x.clamp_min(0.)

In [10]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [11]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [12]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

## Loss function: MSE

In [13]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [14]:
res.squeeze().shape

torch.Size([10000])

In [15]:
y_train, y_valid = y_train.float(), y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [16]:
(y_valid - res.squeeze()).pow(2).mean()

tensor(4154.01)

In [17]:
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [18]:
mse(preds,y_train)

tensor(4308.76)

## Gradients and backward pass

In [19]:
from sympy import symbols, diff
x, y = symbols('x y')
diff(x**2, x)

2*x

In [20]:
diff(x**3, x)

3*x**2

In [21]:
diff(3*x**2+8, x)

6*x

In [22]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t() 
    # w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    w.g = inp.T@out.g
    b.g = out.g.sum(0)

In [23]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0] -targ
    loss = diff.pow(2).mean()

    # backward pass:
    out.g = 2.*diff[:, None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [24]:
forward_and_backward(x_train, y_train)

In [25]:
x_train.g

tensor([[    -0.00,     -0.01,      0.00,  ...,     -0.00,      0.00,      0.00],
        [    -0.03,     -0.03,      0.01,  ...,     -0.04,     -0.01,     -0.01],
        [     0.00,      0.00,     -0.00,  ...,      0.00,     -0.00,      0.00],
        ...,
        [    -0.00,     -0.02,      0.01,  ...,     -0.00,     -0.00,      0.00],
        [    -0.02,     -0.01,      0.01,  ...,     -0.01,      0.01,     -0.00],
        [    -0.00,     -0.00,      0.00,  ...,     -0.00,     -0.00,     -0.00]])

In [26]:
def get_grad(x): return x.g.clone()
chks = w1, w2, b1,b2,x_train
grads = w1g, w2g, b1g, b2g, ig = tuple(map(get_grad, chks))

In [27]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12, w22, b12, b22, xt2 = tuple(map(mkgrad, chks))

In [28]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [29]:
loss = forward(xt2, y_train)
loss.backward()

In [30]:
for a, b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

# Refactor model
## Layers as classes

Dunder call,是指可以直接调用python对象的一种语法糖

In [31]:
from typing import Any


class A:
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        print(f'hi, {args}')

In [32]:
a = A()
a('python')

hi, ('python',)


Python方法中，args和kwds都是函数的参数，但它们在使用上有一些区别。

args是一个用于接收位置参数的元组，可以接受任意数量的位置参数。当调用函数时，传递的位置参数会被收集到args元组中。在函数内部，你可以使用args来访问和处理传递的位置参数。例如，如果使用func(1, 2, 3)来调用函数，那么args将是(1, 2, 3)。

kwds是一个用于接收关键字参数的字典，可以接受任意数量的关键字参数。当调用函数时，传递的关键字参数会被收集到kwds字典中。在函数内部，你可以使用kwds来访问和处理传递的关键字参数。例如，如果使用func(a=1, b=2, c=3)来调用函数，那么kwds将是{'a': 1, 'b': 2, 'c': 3}。

需要注意的是，在函数定义的参数列表中，*args表示接受任意数量的位置参数，**kwds表示接受任意数量的关键字参数。而在函数体内部，args和kwds是实际接收到的位置参数和关键字参数的引用。

In [33]:
from typing import Any

class Relu:

    def __call__(self, inp) -> Any:
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out

    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [34]:
from typing import Any

class Lin:
    def __init__(self, w, b) -> None:
        self.w = w
        self.b = b

    def __call__(self, inp: Any, **kwds: Any) -> Any:
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [35]:
from typing import Any


class Mse:
    def __call__(self, inp, targ) -> Any:
        self.inp, self.targ = inp, targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [36]:
from typing import Any


class Model:
    def __init__(self, w1, b1, w2, b2) -> None:
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()

    def __call__(self, x, targ) -> Any:
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [37]:
model = Model(w1, b1, w2, b2)

In [38]:
loss = model(x_train, y_train)

In [39]:
model.backward()

In [40]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

# Module.forward()

In [41]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [42]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [43]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [44]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [45]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [46]:
model = Model(w1, b1, w2, b2)

In [47]:
loss = model(x_train, y_train)

In [48]:
model.backward()

In [49]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

# Autograd

In [50]:
import torch
from torch import nn
import torch.nn.functional as F

In [51]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out) -> None:
        super().__init__()
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()

    def forward(self, inp): return inp@self.w + self.b

In [53]:
from typing import Any


class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in, nh), nn.ReLU(), Linear(nh, n_out)]

    def __call__(self, x, targ) -> Any:
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:, None])


In [54]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [55]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])